In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs
from datetime import datetime
import random
import re
import pprint
import json
import pandas as pd

def getMatchLinks (articleUrl):
    html = urlopen("http://www.espn.co.uk/rugby/fixtures/_/date/" + articleUrl)
    bsObj = bs(html, "lxml")
    tst = bsObj.find("section", {"id":"pane-main"}).findAll("script", {"type":"text/javascript"})
    for t in tst:
        json_text = re.search(r'^\s*window\.__INITIAL_STATE__\s*=\s*({.*?})\s*;\s*$',
                      t.string, flags=re.DOTALL | re.MULTILINE).group(1)

    json_out = json.loads(json_text)
    
    # Get just the match links
    json_match = json_out['schedule']['groups']
    matches = []
    for m in json_match:
        for c in m['complete']:
            matches.append(c['result']['href'])
            
    return matches

links = getMatchLinks('20160917')
links

['/rugby/report?gameId=289315&league=244293',
 '/rugby/report?gameId=268579&league=244293',
 '/rugby/report?gameId=290006&league=267979',
 '/rugby/report?gameId=290007&league=267979',
 '/rugby/report?gameId=290008&league=267979',
 '/rugby/match?gameId=290268&league=270559',
 '/rugby/match?gameId=290265&league=270559',
 '/rugby/match?gameId=290266&league=270559',
 '/rugby/match?gameId=290270&league=270559',
 '/rugby/match?gameId=290269&league=270559',
 '/rugby/report?gameId=290441&league=270557',
 '/rugby/report?gameId=290440&league=270557',
 '/rugby/match?gameId=290152&league=270555',
 '/rugby/match?gameId=290149&league=270555',
 '/rugby/match?gameId=289875&league=289232',
 '/rugby/match?gameId=290197&league=270563',
 '/rugby/match?gameId=290196&league=270563']

In [2]:
def getStatsLinks (articleUrl):
    html = urlopen("http://www.espn.co.uk" + articleUrl)
    bsObj = bs(html, "lxml")
    tst = bsObj.find("section", {"id":"pane-main"}).findAll("script", {"type":"text/javascript"})
    
    json_out = []
    for t in tst:
        json_text = re.search(r'^\s*window\.__INITIAL_STATE__\s*=\s*({.*?})\s*;\s*$',
                          t.string, flags=re.DOTALL | re.MULTILINE).group(1)
        json_out.append(json.loads(json_text))

    for j in json_out[0]['gamePackage']['links']:
        if j['pageType'] == 'matchstats':
            match_s = j['href']
        elif j['pageType'] == 'playerstats':
            player_s = j['href']
          
    try: 
        match_s, player_s
    except NameError:
        match_s, player_s = None, None

    return match_s, player_s

m, p = getStatsLinks(links[5])
print(m)
print(p)

/rugby/matchstats?gameId=290268&league=270559
/rugby/playerstats?gameId=290268&league=270559


In [3]:
def getStatsLinks (articleUrl):
    html = urlopen("http://www.espn.co.uk" + articleUrl)
    bsObj = bs(html, "lxml")
    tst = bsObj.find("section", {"id":"pane-main"}).findAll("script", {"type":"text/javascript"})
    
    json_out = []
    for t in tst:
        json_text = re.search(r'^\s*window\.__INITIAL_STATE__\s*=\s*({.*?})\s*;\s*$',
                              t.string, flags=re.DOTALL | re.MULTILINE)
        if json_text:
            json_out.append(json.loads(json_text.group(1)))
    
    if len(json_out) > 0:
        for j in json_out[0]['gamePackage']['links']:
            if j['pageType'] == 'matchstats':
                match_s = j['href']
            elif j['pageType'] == 'playerstats':
                player_s = j['href']
          
    try: 
        match_s, player_s
    except NameError:
        match_s, player_s = None, None

    return match_s, player_s

m, p = getStatsLinks(m)
print(m)
print(p)

/rugby/matchstats?gameId=290268&league=270559
/rugby/playerstats?gameId=290268&league=270559


In [4]:
def getStats(statsUrl):
    html = urlopen("http://www.espn.co.uk" + statsUrl)
    bsObj = bs(html, "lxml")  
    tst = bsObj.find("section", {"id":"pane-main"}).findAll("script", {"type":"text/javascript"})
    
    stat_out = []
    for t in tst:
        json_text = re.search(r'^\s*window\.__INITIAL_STATE__\s*=\s*({.*?})\s*;\s*$',
                              t.string, flags=re.DOTALL | re.MULTILINE).group(1)
        stat_out.append(json.loads(json_text))
        
    return stat_out[0]

In [5]:
def getPlayerVals(d):

    dct = d
    dct['match_date'] = date
    dct['match_id'] = match_id[0] # regex above returns list of one element
    if 'id' in dct:
        dct['player_id'] = dct.pop('id') # rename 'id' to 'player_id'
    # This value should be unique and will be the primary key of the table
    dct['row_nk'] = dct['player_id'] + dct['match_id'] 

    # remove entries we don't want - url is useless and eventTimes doesn't conform to the required structure
    # these are the keys, i.e. table column names
    cols = list(dct.keys())
    cols = list(filter(lambda c: c != 'eventTimes', cols))
    cols = list(filter(lambda c: c != 'url', cols))

    # these are used for the SQL query, so everything gets inserted at once
    placeholders = ', '.join(['%s'] * len(cols))
    columns = ', '.join(cols)

    # These are the values to be inserted into the table. Sometimes the values is a single value
    # and other times is a dictionary, handle both instances
    vals = []
    for d in dct:
        if d in cols:
            if type(dct[d]) is dict:
                vals.append(dct[d]['value'])
            else:
                vals.append(dct[d])

    return placeholders, columns, vals

In [6]:
def getMatchVals(m_stats):
    # home team name from player stats
    home = m_stats['gamePackage']['gameStrip']['teams']['home']['name']
    # Away team name from player stats:
    away = m_stats['gamePackage']['gameStrip']['teams']['away']['name']
    # Stadium from player stats:
    ground = m_stats['gamePackage']['matchDetails']['venue']
    # Match date from player stats:
    dt = m_stats['gamePackage']['matchDetails']['date']
    # Competition from player stats:
    comp = m_stats['gamePackage']['gameStrip']['header']
    # Get the date as a date object, from match stats:
    try:
        as_dt = datetime.strptime(dt, '%I:%M PM, %B %d, %Y')
    except ValueError:
        as_dt = datetime.strptime(dt, '%I:%M AM, %B %d, %Y')
    # Exract the date only poriton
    dt_only = as_dt.date()
    # Extract the time portion
    tm = as_dt.time()
    # Year only
    yr = as_dt.year
    # Month only
    mth = as_dt.month
    # Day only
    dy = as_dt.day
    
    return [home, away, ground, dt, comp, as_dt, dt_only, tm, yr, mth, dy]

In [7]:
# Set up the connection
import pymysql
conn = pymysql.connect(host = '127.0.0.1', port = 3306,
                       user = 'root', passwd = '', db = 'mysql')

cur = conn.cursor()

In [34]:
# query to create the table in MySQL - this will need a match and player identifier
sql = """drop table if exists scraping.p_stats;
        create table p_stats( id bigint(10) not null auto_increment
                            , row_nk varchar(50) not null
                            , player_id bigint(7) not null
                            , match_id varchar(20) not null
                            , match_date varchar(20)
                            , name varchar(200)
                            , number varchar(10)
                            , position varchar(10)
                            , captain varchar(10)
                            , subbed varchar(10)
                            , homeAway varchar(10)
                            , subToolTip varchar(10)
                            , onPitch varchar(10)
                            , wasActive varchar(10)
                            , tries bigint(7)
                            , tryassists bigint(7)
                            , points bigint(7)
                            , kicks bigint(7)
                            , passes bigint(7)
                            , runs bigint(7)
                            , metres bigint(7)
                            , cleanbreaks bigint(7)
                            , defendersbeaten bigint(7)
                            , offload bigint(7)
                            , lineoutwonsteal bigint(7)
                            , turnoversconceded bigint(7)
                            , tackles bigint(7)
                            , missedtackles bigint(7)
                            , lineoutswon bigint(7)
                            , penaltiesconceded bigint(7)
                            , yellowcards bigint(7)
                            , redcards bigint(7)
                            , penalties bigint(7)
                            , penaltygoals bigint(7)
                            , conversiongoals bigint(7)
                            , dropgoalsconverted bigint(7)
                            , primary key(id));""".replace("\n", "")

# Run this first to set up connection drop the table, and create again before putting more data into it
cur.execute("USE scraping")
cur.execute(sql)
conn.commit()

In [35]:
# query to create the match details table
msql = """drop table if exists scraping.m_details;
        create table m_details( id bigint(10) not null auto_increment
                                , row_nk varchar(50) not null
                                , home_team varchar (50)
                                , away_team varchar (50)
                                , ground varchar (50)
                                , match_dttime_str varchar(50)
                                , competition varchar(50)
                                , match_dttime varchar(50)
                                , date varchar (20)
                                , time varchar (20)
                                , year varchar (10)
                                , month varchar (5)
                                , day varchar (5) 
                                , primary key(id));""".replace("\n", "")

# Run this first to set up connection drop the table, and create again before putting more data into it
cur.execute("USE scraping")
cur.execute(msql)
conn.commit()

In [8]:
# Function to return the list of match IDs that are already in the table. Reason is so the table can be
# updated without hitting the duplicate key
def getMatches():
    matches = []
    # cur = conn.cursor()
    cur.execute("USE scraping")
    cur.execute("select distinct match_id from p_stats;")
    conn.commit()
    
    numrows = int(cur.rowcount)
    for x in range(0,numrows):
        row = cur.fetchone()
        matches.append(row[0])
    
    return matches

In [9]:
exst_matches = getMatches()
exst_matches
#cur.close()
#conn.close()

['290797',
 '290796',
 '290795',
 '290778',
 '290087',
 '290090',
 '290521',
 '290522',
 '290525',
 '290780',
 '290779',
 '290088',
 '290089',
 '290364',
 '290363',
 '290366',
 '290365',
 '290524',
 '290520',
 '290523',
 '290786',
 '290785',
 '290784',
 '290783',
 '290782',
 '290781',
 '290091',
 '290092',
 '290368',
 '290369',
 '290367',
 '290911',
 '290788',
 '290787',
 '290913',
 '290912',
 '290375',
 '290371',
 '290370',
 '290373',
 '290376',
 '290474',
 '290794',
 '290789',
 '290792',
 '290793',
 '290791',
 '290790',
 '290372',
 '290374',
 '290916',
 '290915',
 '290914',
 '290381',
 '290377',
 '290379',
 '290802',
 '290801',
 '290800',
 '290799',
 '290798',
 '289960',
 '289959',
 '289958',
 '289957',
 '289956',
 '289955',
 '289954',
 '289953',
 '289952',
 '289951',
 '289950',
 '289949',
 '276667',
 '289972',
 '289971',
 '289970',
 '289969',
 '289968',
 '289967',
 '289966',
 '289965',
 '289964',
 '289963',
 '289962',
 '289961',
 '289982',
 '289981',
 '289980',
 '289979',
 '289978',

In [10]:
len(exst_matches)

5032

In [12]:
dt = pd.date_range('20170301','20170430')
#dt = pd.date_range('20160801','20160831')
dates = dt.format(formatter=lambda x: x.strftime('%Y%m%d'))
#dates

In [13]:
# populate the player stats database. This fails sometimes for reasons I can't understand
# if it does, need to restart the Apache DB and probably drop the table, and try again
teams, players = ['home', 'away'], ['team', 'reserves']
# dates = ['20170304', '20170305'] # Testing with two dates
# dates = ['20160917']
not_captured = []

for date in dates:
    print(date)
    matches = getMatchLinks(date)
    
    if len(matches) > 0:
        for match_link in matches:
            if match_link != '#':
                try:
                    ma, pl = getStatsLinks(match_link)
                except KeyError:
                    continue

                regex = re.compile('gameId=([0-9]*)')
                match_id = regex.findall(match_link)
                
                if ma == None and pl == None:
                    not_captured.append(match_id)
                
                # Only run the code if the match is not already in the database
                if match_id[0] not in exst_matches:
                
                    # Update the match details table
                    if ma != None:
                        m_stats = getMatchVals(getStats(ma))
                        m_stats.insert(0, match_id[0])

                        columns = ['row_nk', 'home_team', 'away_team', 'ground','match_dttime_str', 'competition', 
                            'match_dttime', 'date', 'time', 'year', 'month', 'day']
                        cols = 'row_nk, home_team, away_team, ground, match_dttime_str, competition, match_dttime, date, time, year, month, day'
                        placeholders = ', '.join(['%s'] * len(columns))

                        #cur = conn.cursor()
                        cur.execute("USE scraping;")

                        # Execute the SQL to update the table
                        sql_update = "insert into m_details (%s) values (%s);" % (cols, placeholders)
                        cur.execute(sql_update, m_stats)
                        conn.commit()

                    # Update the player stats table
                    if pl != None:
                        p_stats = getStats(pl)

                        for t in teams:
                            for p in players:
                                data = p_stats["gamePackage"]["matchLineUp"][t][p]
                                for d in data:               
                                    placeholders, columns, vals = getPlayerVals(d)

                                    #cur = conn.cursor()
                                    cur.execute("USE scraping")

                                    # Execute the SQL to update the table
                                    sql_update = "insert into p_stats (%s) values (%s)" % (columns, placeholders)
                                    cur.execute(sql_update, vals)
                                    conn.commit()

# cur.close()
# conn.close()

20170301
20170302
20170303
20170304
20170305
20170306
20170307
20170308
20170309
20170310
20170311
20170312
20170313
20170314
20170315
20170316
20170317
20170318
20170319
20170320
20170321
20170322
20170323
20170324
20170325
20170326
20170327
20170328
20170329
20170330
20170331
20170401
20170402
20170403
20170404
20170405
20170406
20170407
20170408
20170409
20170410
20170411
20170412
20170413
20170414
20170415
20170416
20170417
20170418
20170419
20170420
20170421
20170422
20170423
20170424
20170425
20170426
20170427
20170428
20170429
20170430


In [43]:
# Run this if the code fails to close the DB connection
cur.close()
conn.close()

In [92]:
match_link

'/rugby/report?gameId=241617&league=242041'

In [78]:
html = urlopen("http://www.espn.co.uk" + match_link)
bsObj = bs(html, "lxml")
tst = bsObj.find("section", {"id":"pane-main"}).findAll("script", {"type":"text/javascript"})

In [82]:
for t in tst:
    json_text = re.search(r'^\s*window\.__INITIAL_STATE__\s*=\s*({.*?})\s*;\s*$',
                          t.string, flags=re.DOTALL | re.MULTILINE)
    if json_text:
        json_text = json_text.group(1)
    else:
        json_text = re.search(r'^\s*window\.__INITIAL_STATE__\s*=\s*({.*?})\s*;\s*$',
                          str(bsObj), flags=re.DOTALL | re.MULTILINE).group(1).replace("\\", "")
        #json_out.append(json.loads(json_text))

In [99]:
not_captured

[['241617'],
 ['240011'],
 ['240013'],
 ['256459'],
 ['240017'],
 ['240019'],
 ['240023'],
 ['240021'],
 ['257185'],
 ['262839'],
 ['240033'],
 ['240025'],
 ['240027'],
 ['240029'],
 ['240031'],
 ['242935'],
 ['256461'],
 ['240035'],
 ['240037'],
 ['256463'],
 ['240041'],
 ['242939'],
 ['262849'],
 ['262851'],
 ['262853'],
 ['262855'],
 ['289825'],
 ['262859'],
 ['257187'],
 ['289824'],
 ['242941'],
 ['242943'],
 ['264145'],
 ['264143'],
 ['264141'],
 ['264151'],
 ['264149'],
 ['264147'],
 ['264155'],
 ['264157'],
 ['264159'],
 ['264155'],
 ['264157'],
 ['264159'],
 ['241749'],
 ['289823'],
 ['267295']]

In [16]:
p_stats['gamePackage']

{'HeadToHeadNode': [{'events': [{'atVs': '@',
     'awayTeamId': '25889',
     'awayTeamScore': '15',
     'gameDate': '2016-07-08T07:35Z',
     'gameResult': 'L',
     'homeTeamId': '25932',
     'homeTeamScore': '40',
     'id': '276619',
     'leagueAbbreviation': 'Super Rugby',
     'leagueName': 'Super Rugby',
     'links': [{'href': 'http://www.espn.co.uk/rugby/match?gameId=276619&league=242041',
       'text': 'Summary'},
      {'href': 'http://www.espn.co.uk/rugby/match?gameId=276619&league=242041',
       'text': 'Match'},
      {'href': 'http://www.espn.co.uk/rugby/matchstats?gameId=276619&league=242041',
       'text': 'Match Stats'}],
     'opponent': {'abbreviation': 'BLUES',
      'displayName': 'Blues',
      'id': '25932',
      'links': [{'href': 'http://www.espn.co.uk/rugby/team/_/id/25932/blues',
        'text': 'Clubhouse'}],
      'logo': 'http://a.espncdn.com/i/teamlogos/rugby/teams/500/25932.png',
      'uid': 's:300~t:25932'},
     'opponentLogo': 'http://a.espn